In [1]:
import numpy as np
import pandas as pd
from pytube import YouTube
import os
os.environ["IMAGEIO_FFMPEG_EXE"] = "/usr/bin/ffmpeg"
import moviepy.editor as mp


In [2]:
data = pd.read_csv("url.csv")


In [3]:
target_path = "./mp3_data/mp3"
traintxt = open("./mp3_data/train.txt", "w")
traintxt.write("\"file\",\"Description\"\n")

cant_downlaod = []
cant_rename = []

for i in range(len(data)):
    url = "https://www.youtube.com/watch?v=" + data["ID"][i]

    yt = YouTube(url = url, use_oauth=True, allow_oauth_cache=True)


    try: 
        video = yt.streams.filter(only_audio=True).first()
        out_file = video.download(output_path=target_path)
    except:
        print(data["Title"][i], "cant downlaod")
        cant_downlaod.append(i)
        continue

    name = data["Title"][i].replace(".", "").replace("\"", "")
    
    try:
        os.rename("./mp3_data/mp3/" + name + ".mp4", "./mp3_data/mp3/" + str(i) + ".mp4")
    except:
        print(data["Title"][i], "cant rename")
        cant_rename.append(i)
        continue

    # video = mp.VideoFileClip( "./mp3_data/mp3/" + str(i) + ".mp4")
    # video.audio.write_audiofile("./mp3_data/mp3/" + str(i) + ".mp3")

print("cant_downlaod: ", cant_downlaod)
print("cant_rename: ", cant_rename)

traintxt.close()


石虎生存危機 光電.生態.國土規畫如何兼顧【客家新聞20230719】
石虎生存危機 光電生態國土規畫如何兼顧【客家新聞20230719】
拜訪竹苗移居南非鄉親 看見立足天涯智謀【客家新聞20230718】
拜訪竹苗移居南非鄉親 看見立足天涯智謀【客家新聞20230718】
友善農耕護水雉 野蓮農投入生態保育計畫【客家新聞20230718】
友善農耕護水雉 野蓮農投入生態保育計畫【客家新聞20230718】
剪黏泥塑技術保存者徐明河辭世 享壽82歲【客家新聞20230718】
剪黏泥塑技術保存者徐明河辭世 享壽82歲【客家新聞20230718】
新瓦屋板凳電影院將登場 抽紀念摺疊板凳【客家新聞20230718】
新瓦屋板凳電影院將登場 抽紀念摺疊板凳【客家新聞20230718】
未滿18歲打工 須代理人同意.年齡證明【客家新聞20230718】
未滿18歲打工 須代理人同意年齡證明【客家新聞20230718】
屏東縣設5處智慧回收機 資源回收送咖啡【客家新聞20230718】
屏東縣設5處智慧回收機 資源回收送咖啡【客家新聞20230718】
靜思湖科學人營隊 為學子創造飛夢園【客家新聞20230718】
靜思湖科學人營隊 為學子創造飛夢園【客家新聞20230718】
頭份建國國小合唱團 韓國世界賽奪金.銀牌【客家新聞20230718】
頭份建國國小合唱團 韓國世界賽奪金銀牌【客家新聞20230718】
客委會攜手學者 發表新書"乙未中彰保衛戰"【客家新聞20230718】
客委會攜手學者 發表新書乙未中彰保衛戰【客家新聞20230718】
中寮生態農場排廢染河川 自救會開協調會【客家新聞20230718】
中寮生態農場排廢染河川 自救會開協調會【客家新聞20230718】
南部非洲仲裁協會 臺商經商若遇糾紛首選【客家新聞20230718】
南部非洲仲裁協會 臺商經商若遇糾紛首選【客家新聞20230718】
民生消費產業缺工創新高 專家籲多元徵才【客家新聞20230718】
民生消費產業缺工創新高 專家籲多元徵才【客家新聞20230718】
客家傳統小吃研習 五彩豬籠粄添課程趣味【客家新聞20230717】
客家傳統小吃研習 五彩豬籠粄添課程趣味【客家新聞20230717】
屏東可可揚名國際 ICA亞太區競賽奪7金【客家新聞20230717】
屏東可可揚名國際 IC